In [1]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def AMCS(tr_data, tr_label, ts_data, ts_label):
    U = np.unique(tr_label)  # class labels
    cn = len(U)  # number of classes

    label_class = {}
    size_class = np.zeros(cn)
    for i in range(cn):
        label_class[i] = np.where(ts_label == U[i])[0]
        size_class[i] = label_class[i].shape[0]

    IR = np.max(size_class) / np.min(size_class)
    trnum, dim = tr_data.shape

    if IR >= 10 and cn >= 6 and dim >= 10:
        FAC(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR >= 10 and dim < 10 and cn >= 6:
        BAK(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR >= 10 and dim >= 10 and cn < 6:
        FAC(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR >= 10 and dim < 10 and cn < 6:
        BAS(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR < 10 and dim >= 10 and cn >= 6:
        BAR(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR < 10 and dim < 10 and cn >= 6:
        FAS(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR < 10 and dim >= 10 and cn < 6:
        BAC(tr_data, tr_label, ts_data, ts_label, cn)
    elif IR < 10 and dim < 10 and cn < 6:
        BAS(tr_data, tr_label, ts_data, ts_label, cn)


def BAK(tr_data, tr_label, ts_data, ts_label, cn):
    # Feature selection using BPSO algorithm
    # Implement BPSO algorithm here

    # Ensemble framework using AdaBoost
    classifier = AdaBoostClassifier(base_estimator=KNeighborsClassifier())
    classifier.fit(tr_data, tr_label)
    predictions = classifier.predict(ts_data)

    # Evaluate accuracy
    accuracy = accuracy_score(ts_label, predictions)
    print("Accuracy:", accuracy)



In [2]:

def BPSO(BPSO_num, cn, lab_tr, lab_ts, train_data, test_data):
    # Initial Parameters for PSO
    Max_iteration = 10  # Maximum number of iterations
    noP = 20  # Number of particles
    D = train_data.shape[1]  # Number of dimensions
    w = 2  # Inertia weight
    wMax = 0.9  # Max inertia weight
    wMin = 0.4  # Min inertia weight
    c1 = 2
    c2 = 2
    Vmax = 6

    Velocity = np.zeros((noP, D))  # Velocity vector
    Position = np.zeros((noP, D))  # Position vector
    fitness = np.zeros(noP)

    # Cognitive component
    pBestScore = np.ones(noP)
    pBest = np.ones((noP, D))

    # Social component
    gBestScore = np.inf
    gBest = np.ones(D)

    ConvergenceCurve = np.zeros(Max_iteration)  # Convergence vector

    # Initialization
    for i in range(Position.shape[0]):  # For each particle
        for j in range(Position.shape[1]):  # For each variable
            if np.random.rand() <= 0.5:
                Position[i, j] = 0
            else:
                Position[i, j] = 1

        if np.array_equal(Position[i, :], np.zeros(D)):
            Position[i, np.random.randint(0, D)] = 1
        elif np.array_equal(Position[i, :], np.ones(D)):
            Position[i, np.random.randint(0, D)] = 0

    for l in range(Max_iteration):
        # Calculate cost for each particle
        for i in range(Position.shape[0]):
            fitness[i] = CostFunction(Position[i, :], lab_tr, lab_ts, train_data, test_data)

            if pBestScore[i] > fitness[i]:
                pBestScore[i] = fitness[i]
                pBest[i, :] = Position[i, :]

            if gBestScore > fitness[i]:
                gBestScore = fitness[i]
                gBest = Position[i, :]

        # Update the w of PSO
        w = wMax - l * ((wMax - wMin) / Max_iteration)

        # Update the Velocity and Position of particles
        for i in range(Position.shape[0]):
            for j in range(Position.shape[1]):
                Velocity[i, j] = w * Velocity[i, j] + c1 * np.random.rand() * (pBest[i, j] - Position[i, j]) + c2 * np.random.rand() * (gBest[j] - Position[i, j])

                if Velocity[i, j] > Vmax:
                    Velocity[i, j] = Vmax
                if Velocity[i, j] < -Vmax:
                    Velocity[i, j] = -Vmax

                if BPSO_num == 1:
                    s = 1 / (1 + np.exp(-2 * Velocity[i, j]))  # S1 transfer function
                elif BPSO_num == 2:
                    s = 1 / (1 + np.exp(-Velocity[i, j]))  # S2 transfer function
                elif BPSO_num == 3:
                    s = 1 / (1 + np.exp(-Velocity[i, j] / 2))  # S3 transfer function
                elif BPSO_num == 4:


SyntaxError: unexpected EOF while parsing (<ipython-input-2-90f17dd80ce0>, line 72)

In [ ]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import SelectKBest, chi2

def BAK(train_data, lab_tr, test_data, lab_ts, cn):
    # Perform feature selection using BPSO
    # BPSO code is assumed to be implemented separately

    # gBest contains the selected features
    gBest, _ = BPSO(1, cn, lab_tr, lab_ts, train_data, test_data)

    # Apply selected features to the training and test data
    train_data_selected = train_data[:, gBest]
    test_data_selected = test_data[:, gBest]

    # Train Adaboost classifier using KNN as the base classifier
    base_classifier = KNeighborsClassifier()
    adaboost = AdaBoostClassifier(base_classifier, n_estimators=50)
    adaboost.fit(train_data_selected, lab_tr)

    # Predict labels for test data
    hypo = adaboost.predict(test_data_selected)

    # Calculate accuracy and AUC area
    acc = accuracy_score(lab_ts, hypo)
    auc_area = roc_auc_score(lab_ts, hypo)

    # Calculate GM and FV metrics
    gm, fv = G(lab_ts, hypo, cn)

    return hypo, acc, auc_area, gm, fv


def G(y_true, y_pred, cn):
    # GM and FV calculation code is assumed to be implemented separately
    # y_true: true labels
    # y_pred: predicted labels
    # cn: number of classes

    # Placeholder code for calculating GM and FV
    gm = 0.0
    fv = 0.0

    return gm, fv
